Вашим заданием в этом модуле будет проанализировать и преобразовать данные о велопоездках клиентов компании Citi Bike (США), специализирующейся на прокате велосипедов.
Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

        starttime — время начала поездки (дата, время);
        stoptime — время окончания поездки (дата, время);
        start station id — идентификатор стартовой стоянки;
        start station name — название стартовой стоянки;
        start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
        end station id — идентификатор конечной стоянки;

        end station name — название конечной стоянки;
        end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
        bikeid — идентификатор велосипеда;
        usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
        birth year — год рождения клиента;
        gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).



In [4]:
import pandas as pd
bike_data = pd.read_csv('data/citibike-tripdata.csv', sep =',')
bike_df = bike_data.copy()

In [8]:
bike_df.info()
# start station id missed


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

## Задание 6.3
Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.

In [11]:
bike_df['start station id'].value_counts().nlargest(1)

281.0    1928
Name: start station id, dtype: int64

##  Задание 6.4
Велосипед с каким идентификатором является самым популярным?

In [12]:

bike_df['bikeid'].value_counts().nlargest(1)

33887    107
Name: bikeid, dtype: int64

##  Задание 6.5
Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.

In [13]:
bike_df['usertype'].value_counts(normalize='True')

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

##  Задание 6.6 
Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.

In [22]:
bike_df['gender'].value_counts().max()

1

##  Задание 6.7
Выберите утверждения,которые соответствуют нашим данным:
A: Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу
B: В рассматриваемые дни минимальный возраст клиента составлял 10 лет
C: Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave"
D: Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail" 

In [27]:
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
display(bike_df['starttime'].dt.year.min())


2018

In [43]:
print('A:', bike_df['start station name'].unique() != bike_df['end station name'].unique())
print('B:', (bike_df['starttime'].dt.year.min() - bike_df['birth year'].max())<10)
print('C:', bike_df['start station name'].value_counts().index[-1] == "Eastern Pkwy & Washington Ave")
print('D:', bike_df['end station name'].value_counts().index[0] == "Liberty Light Rail")

A: True
B: False
C: True
D: False


/tmp/ipykernel_50430/682364967.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print('A:', bike_df['start station name'].unique() != bike_df['end station name'].unique())


## Задание 6.8
В первую очередь удалим лишнюю информацию из данных.

В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки. Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?

In [46]:
bike_df.drop(['start station id', 'end station id'], axis = 1, inplace = True)

KeyError: "['start station id' 'end station id'] not found in axis"

In [51]:
len(bike_df.columns)

12

##  Задание 6.9
Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?

In [52]:
bike_df['age'] = 2018 - bike_df['birth year']

In [58]:
display(bike_df[bike_df['age']>60]['age'].shape[0])

11837

##  Задание 6.10
Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки (stoptime) и временем её начала (starttime) в секундах. В качестве ответа запишите среднюю длительность поездки в секундах. Ответ округлите до целого.

Для того чтобы преобразовать временной интервал timedelta в секунды, используется атрибут dt.seconds (в datetime атрибут был second — в единственном числе).

In [64]:
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])
bike_df[['starttime', 'stoptime']]

,starttime,stoptime
0,2018-09-01 00:00:05.269,2018-09-01 00:27:20.634
1,2018-09-01 00:00:11.281,2018-09-01 00:02:23.481
2,2018-09-01 00:00:20.649,2018-09-01 00:55:58.547
3,2018-09-01 00:00:21.746,2018-09-01 00:07:38.583
4,2018-09-01 00:00:27.315,2018-09-01 02:21:25.308
...,...,...
299995,2018-09-05 19:08:27.846,2018-09-05 19:15:51.494
299996,2018-09-05 19:08:28.370,2018-09-05 19:20:01.508
299997,2018-09-05 19:08:27.509,2018-09-05 19:13:40.506
299998,2018-09-05 19:08:29.230,2018-09-05 20:04:29.322


In [69]:
bike_df['trip duration'] = (bike_df['stoptime'] - bike_df['starttime']).\bike_df['trip duration']

AttributeError: 'Series' object has no attribute 'seconds'